In [11]:
import pandas as pd
import plotly.express as px
from google.oauth2 import service_account
from google.cloud import bigquery
import streamlit as st
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# En este mundo de las IDEs hay que actualizarse
Esto es una prueba de markdown en pycharm. Por cierto, la ide es una pasada.

In [14]:
# Create API client.
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"]
)
client = bigquery.Client(credentials=credentials)

# Perform query.
# Uses st.cache to only rerun when the query changes or after 10 min.
#@st.cache(ttl=600)
def run_query():
    df_1 = client.query("select * from psycho_research.df_scales").to_dataframe()
    df_2 = client.query("select * from psycho_research.df_demographics").to_dataframe()
    df = pd.merge(df_1, df_2, how = 'left', left_on = ['email', 'Generation_key'], right_on = ['email', 'Generation_key'])
    #me quedo sólo con los completos:
    df = df.loc[df.Completion_prepost == 'Completed']
    return df

df = run_query()
df

,id_x,email,Form,Generation_key,Completion_prepost,DASS_depression,DASS_anxiety,DASS_stress,DASS_global,FFMQ_act,...,Meditation_since_months,Meditation_interrupted_months,Meditation_frequency_weekly,Meditation_totaltime_daily,Informal_practice_totaltime_daily,Meditation_last_time_hours,Meditation_retire,Meditation_retire_amount,Meditation_retire_totaldays,Meditation_retire_last_time_months
0,89,lauriyya@yahoo.es,Formulario Pre,MBSR_2017_Agustin_2_X_m_p,Completed,24.0,2.0,2.0,28.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
1,93,nicezucchini@hotmail.com,Formulario Pre,MBSR_2017_Gustavo_2_M_t_p,Completed,24.0,18.0,36.0,78.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
2,95,raquel.penarando@gmail.com,Formulario Pre,MBSR_2017_Gustavo_2_M_t_p,Completed,6.0,4.0,14.0,24.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
4,102,lozanosuarezb@gmail.com,Formulario Pre,MBSR_2017_Agustin_2_X_m_p,Completed,12.0,2.0,10.0,24.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
6,108,carrillocris@gmail.com,Formulario Pre,CCT_2017_Gonzalo_2_X_m_p,Completed,22.0,2.0,12.0,36.0,17.0,...,25.0,1.0,7.0,45.0,45.0,2.0,Sí,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,1117,palomagilardi@gmail.com,Formulario Pre,CCT_2021_Silvia_2_X_m_o,Completed,12.0,16.0,20.0,48.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
1580,1078,martadoble@gmail.com,Formulario Pre,MBSR_2021_Agustin_1_X_t_o,Completed,16.0,10.0,24.0,50.0,8.0,...,24.0,12.0,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
1581,895,lauramestre@gmail.com,Formulario Post,MBSR_2019_Agustin_2_M_t_p,Completed,0.0,0.0,4.0,4.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
1582,1056,morenoisa@telefonica.net,Formulario Pre,MBSR_2018_Gustavo_2_M_t_p,Completed,2.0,4.0,16.0,22.0,17.0,...,144.0,36.0,1.0,30.0,0.0,48.0,Sí,1.0,2.0,120.0


In [33]:

posibles_comparaciones = ['Format', 'Teacher', 'Gender', None]
lista_escalas = ['DASS_stress', 'DASS_anxiety', 'DASS_depression']
for escala in lista_escalas:
    height = len(df['Format'].unique()) * 400
    fig = px.box(df, x="Program", y=escala, points='all',
             color = "Form" , title=escala + ' by Format', notched=True,
             facet_row = "Format",
             color_discrete_sequence=px.colors.qualitative.Pastel, template="plotly_dark",
             height= height
            )
    fig.show()



In [17]:
lista_escalas = ['DASS_depression', "DASS_anxiety"]
data = df.loc[df.scales.isin(lista_escalas)]
data /= 1000000.0
data = data.T.reset_index()
data = pd.melt(data, id_vars=["index"]).rename(columns={"index": "year", "value": "Gross Agricultural Product ($B)"})
chart = (alt.Chart(data)
        .mark_area(opacity=0.3)
        .encode(
            x="year:T",
            y=alt.Y("Gross Agricultural Product ($B):Q", stack=None),
            color="Region:N",
        )
        )

['DERS_confusion',
 'DERS_desatencion',
 'DERS_interferencia',
 'DERS_rechazo',
 'DERS_descontrol',
 'DERS_global',
 'CS_kindness',
 'CS_common_humanity',
 'CS_mindfulness',
 'CS_indifference',
 'CS_global',
 'DASS_depression',
 'DASS_anxiety',
 'DASS_stress',
 'DASS_global',
 'MAIA_noticing',
 'MAIA_not_distracting',
 'MAIA_not_worrying',
 'MAIA_att_regulation',
 'MAIA_em_regulation',
 'MAIA_self_regulation',
 'MAIA_body_listening',
 'MAIA_trusting',
 'PHI_social_wb',
 'FFMQ_nonreact',
 'SCS_self_kindness',
 'SCS_self_judgment',
 'SCS_common_humanity',
 'SCS_isolation',
 'SCS_mindfulness',
 'SCS_over_identified',
 'SCS_global',
 'EQ',
 'PHI_general_wb',
 'PHI_eudaimonic_wb',
 'PHI_hedonic_wb',
 'FFMQ_observe',
 'RRS_brooding',
 'SWLS',
 'FFMQ_act',
 'FFMQ_nonjudment',
 'FFMQ_describe',
 'FFMQ_global',
 'MAIA_global']